In [ ]:
from contextlib import closing
from ipywidgets import interact
from ipywidgets import Button
from ipywidgets import Output
from IPython.display import display
from IPython.display import Video
from IPython.display import HTML
from matplotlib import pyplot as plt
from textwrap import dedent
from tqdm.notebook import tqdm
import cv2
import ffmpeg
import imutils
import ipywidgets as widgets
import numpy as np
import tensorflow as tf

In [ ]:
def tf_to_cv(image):
    return (image.numpy().squeeze() * 255.).astype(np.uint8)


def cv_to_tf(image):
    return (tf.cast(image, tf.float32) / 255.)[:, :, tf.newaxis]


def arrange_images(*rows):
    """Combine a table (list of list of image ndarrays) into a single image ndarray."""
    rows = [[tf_to_cv(col) for col in row] for row in rows]
    blank_image = np.ones(rows[0][0].shape, dtype=np.uint8)
    col_count = max(len(row) for row in rows)
    rows = [list(row) + [blank_image] * (col_count - len(row)) for row in rows]
    return cv_to_tf(np.vstack(list(np.hstack(row) for row in rows)))


def load_video(in_filename):
    probe = ffmpeg.probe(in_filename)
    video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
    width = int(video_info['width'])
    height = int(video_info['height'])
    num_frames = int(video_info['nb_frames'])
    out, err = (
        ffmpeg.input(in_filename)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24')
        .run(capture_stdout=True)
    )
    return cv_to_tf(np.frombuffer(out, np.uint8).reshape([-1, height, width, 3]))


class AutoplayVideo(Video):
    def __init__(
        self,
        data=None,
        url=None,
        filename=None,
        embed=False,
        mimetype=None,
        width=None,
        height=None,
        controls=True,
        autoplay=True,
        loop=True,
    ):
        super(AutoplayVideo, self).__init__(
            data, url, filename, embed, mimetype, width, height
        )
        self.controls = controls
        self.autoplay = autoplay
        self.loop = loop

    def _repr_html_(self):
        assert not self.embed, 'Embedding not implemented (yet)'
        options = []
        if self.width:
            options.append('width={}'.format(self.width))
        if self.height:
            options.append('height={}'.format(self.height))
        if self.autoplay:
            options.append('autoplay')
        if self.controls:
            options.append('controls')
        if self.loop:
            options.append('loop')
        url = self.url if self.url is not None else self.filename
        disclaimer = 'Your browser does not support the <code>video</code> element.'
        return '<video src="{}" {}>{}</video>'.format(
            url, ' '.join(options), disclaimer
        )


def erode(image, erosion, iterations=1):
    if erosion:
        kernel = np.ones((erosion, erosion), np.uint8)
        image = cv_to_tf(cv2.erode(tf_to_cv(image), kernel, iterations=iterations))
    return image


def dilate(image, dilation, iterations=1):
    if dilation:
        kernel = np.ones((dilation, dilation), np.uint8)
        image = cv_to_tf(cv2.dilate(tf_to_cv(image), kernel, iterations=iterations))
    return image


def blur(image, blurriness):
    if blurriness:
        image = cv_to_tf(cv2.blur(tf_to_cv(image), (blurriness, blurriness)))
    return image

In [ ]:
in_filename = 'vid3_small.mp4'
frames = load_video(in_filename)

In [ ]:
# frames = frames[:20]
out_filename = 'out.mp4'

do_render = lambda: None

bgs = []
back_sub = cv2.createBackgroundSubtractorMOG2()
for frame in frames:
    bgs.append(cv_to_tf(back_sub.apply(tf_to_cv(frame))))


frame_count = len(frames)

def match_hsv(image, lower_hsv, upper_hsv, blurriness=2, dilation=30):
    hsv_image = tf.image.rgb_to_hsv(image)
    hsv_image = blur(hsv_image, blurriness)
    mask = cv_to_tf(cv2.inRange(tf_to_cv(hsv_image), lower_hsv.numpy() * 255, upper_hsv.numpy() * 255))
    mask = dilate(mask, dilation)
    return mask


@interact(
    preview_frame_num=(1, frame_count - 1),
    end_frame_num=(1, frame_count - 1),
    h1=(0, 255, 1),
    s1=(0, 255, 1),
    v1=(0, 255, 1),
    h2=(0, 255, 1),
    s2=(0, 255, 1),
    v2=(0, 255, 1),
    blurriness=(0, 20),
    dilation1=(0, 80, 1),
    dilation2=(0, 80, 1),
    dilation3=(0, 80, 1),
    min_radius=(0, 40, 1),
    max_radius=(0, 40, 1),
)
def show_frame(
    preview_frame_num=97,
    end_frame_num=frame_count - 1,
    h1=0,
    s1=0,
    v1=216,
    h2=39,
    s2=42,
    v2=255,
    blurriness=2,
    dilation1=22,
    dilation2=60,
    dilation3=10,
    min_radius=12,
    max_radius=30,
):
    def render_frame(frame_num):
        lower_hsv = tf.constant([h1 / 255, s1 / 255, v1 / 255])
        upper_hsv = tf.constant([h2 / 255, s2 / 255, v2 / 255])
        mask1 = match_hsv(frames[frame_num], lower_hsv, upper_hsv, blurriness, dilation1)
        mask2 = match_hsv(frames[frame_num - 1], lower_hsv, upper_hsv, blurriness, dilation2)
        mask3 = tf.cast(tf.cast(mask1, tf.bool) & ~tf.cast(mask2, tf.bool), tf.float32)
        mask3 = dilate(mask3, dilation3)
        mask1_rgb = tf.image.grayscale_to_rgb(mask1)
        mask2_rgb = tf.image.grayscale_to_rgb(mask2)
        mask3_rgb = tf.image.grayscale_to_rgb(mask3)

        contours, _ = cv2.findContours(tf_to_cv(mask3), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        final_image = tf_to_cv(frames[frame_num])
        for contour in contours:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            if min_radius <= radius <= max_radius:
                final_image = cv2.circle(final_image, (int(x), int(y)), int(radius), (0, 255, 0), 2)
        final_image = cv_to_tf(final_image)

        return arrange_images(
            (
                #frames[frame_num],
                final_image,
                mask1_rgb,
            ),
            (
                mask2_rgb,
                mask3_rgb,
            ),
        )

    out_image = render_frame(preview_frame_num)
    fig, ax = plt.subplots(1, figsize=(16, 9))
    fig.tight_layout()
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(tf_to_cv(out_image))
    out_height, out_width = out_image.shape[:2]

    def _render():
        print('Rendering...')
        process = (
            ffmpeg.input(
                'pipe:',
                format='rawvideo',
                pix_fmt='rgb24',
                s='{}x{}'.format(out_width, out_height),
            )
            .filter('setpts', '1.4*PTS')
            .output(out_filename, pix_fmt='yuv420p')
            .overwrite_output()
            .run_async(pipe_stdin=True)
        )
        with closing(process.stdin) as pipe:
            for frame_num in tqdm(range(1, end_frame_num)):
                out_image = render_frame(frame_num)
                pipe.write(tf_to_cv(out_image).astype(np.uint8).tobytes())
        process.wait()
        print('Done.')
        display(AutoplayVideo(out_filename, width=1400))

    global do_render
    do_render = _render


def render(_):
    with out:
        out.clear_output()
        do_render()


out = Output()
render_button = Button(description='Render')
render_button.on_click(render)
display(render_button)
display(out)

In [ ]:
x = tf.image.rgb_to_grayscale(frames[0])

#plt.imshow(x.numpy().squeeze(), cmap='gray')
@interact(thresh=(0., 1., 0.05))
def f(thresh=0.5):
    plt.figure(figsize=(16, 9))
    plt.imshow(tf.where(x > thresh, tf.ones(x.shape), tf.zeros(x.shape)).numpy().squeeze(), cmap='gray')